In [1]:
import sys
sys.path.append('../')
from onemap_client import OneMapClient
import os
from dotenv import load_dotenv
load_dotenv()
import json
import pandas as pd

email = os.environ['ONE_MAP_API_EMAIL']
password = os.environ['ONE_MAP_API_PASSWORD']

In [2]:
os.makedirs('./amenities', exist_ok=True)

In [3]:
# Initialise Client and retrieve access token
client = OneMapClient(email,password)
access_token = client.get_token(email,password)[0]

# Themes to consider

- Libraries
- Community Clubs

- safra centres
- studentcare
- healtherdining

In [4]:
client.get_all_themes_info(more_info=True)

{'Theme_Names': [{'THEMENAME': 'After Death Facilities',
   'QUERYNAME': 'afterdeathfacilities',
   'ICON': 'coffina20.jpg',
   'EXPIRY_DATE': '31/12/2028',
   'PUBLISHED_DATE': '01/01/2012',
   'CATEGORY': 'Environment',
   'THEME_OWNER': 'NATIONAL ENVIRONMENT AGENCY'},
  {'THEMENAME': 'Kindergartens',
   'QUERYNAME': 'kindergartens',
   'ICON': 'school.gif',
   'EXPIRY_DATE': '31/12/2026',
   'PUBLISHED_DATE': '01/01/2012',
   'CATEGORY': 'Education',
   'THEME_OWNER': 'EARLY CHILDHOOD DEVELOPMENT AGENCY'},
  {'THEMENAME': 'Community Clubs',
   'QUERYNAME': 'communityclubs',
   'ICON': 'PA.jpg',
   'EXPIRY_DATE': '30/12/2022',
   'PUBLISHED_DATE': '31/12/2011',
   'CATEGORY': 'Community',
   'THEME_OWNER': "PEOPLE'S ASSOCIATION"},
  {'THEMENAME': 'Private Education Institutions',
   'QUERYNAME': 'cpe_pei_premises',
   'ICON': 'pink.jpg',
   'EXPIRY_DATE': '30/04/2023',
   'PUBLISHED_DATE': '01/01/2012',
   'CATEGORY': 'Education',
   'THEME_OWNER': 'SKILLSFUTURE SINGAPORE'},
  {'THEM

# Education

## Pre-Schools/Kindergarten

- retrieve_theme endpoint
    - queryname --> 'preschools_location'

In [5]:

search_results = client.retrieve_theme(query_name='preschools_location')['SrchResults']
# search results starts from index
pre_school_locations = {'results':[]}
lat_longs = set()

for pre_school in search_results[1:]:
    coord_list = pre_school['LatLng'].split(',')
    lat_long = tuple([float(coord_list[0]), float(coord_list[1])])
    if lat_long not in lat_longs:
        lat_longs.add(lat_long)
        pre_school_locations['results'].append({'name':pre_school['NAME'],'lat':float(coord_list[0]),'long':float(coord_list[1])})


In [6]:
len(pre_school_locations['results'])

1785

In [7]:
with open('./amenities/preschool_coords.json','w')as f:
    json.dump(pre_school_locations,f)

- retrieve_theme endpoint
    - queryname --> 'kindergartens'

In [9]:
search_results = client.retrieve_theme(query_name='kindergartens')['SrchResults']
# search results starts from index
kind_loc = {'results':[]}
lat_longs = set()
for kind in search_results[1:]:
    coord_list = kind['LatLng'].split(',')
    lat_long = tuple([float(coord_list[0]), float(coord_list[1])])
    if lat_long not in lat_longs:
        lat_longs.add(lat_long)
        kind_loc['results'].append({'name':kind['NAME'],'lat':float(coord_list[0]),'long':float(coord_list[1])})

In [10]:
len(kind_loc['results'])

382

In [11]:
with open('./amenities/kindergarten_coords.json','w')as f:
    json.dump(kind_loc,f)

In [12]:
import re

def filter_school_search(cur_sch_name,existing_sch_list):
   # conditions to check if search result returned a specific block/building within a school or if school has been stored in existing_schools already
   if '@' in cur_sch_name:
      return True
   if '(U/C)' in cur_sch_name:
      return True
   if 'BUILDING' in cur_sch_name:
      return True
   for existing_sch in existing_sch_list:
      if cur_sch_name == existing_sch['name']:
         return True
   pattern = r"(\(|\[)*.(PRIMARY|HIGH|SECONDARY) SCHOOL(\]|\))"
   if re.search(pattern, cur_sch_name):
      return True
   return False

## Primary

- search endpoint
    - keywords --> Primary School

In [20]:
tot_page_num = client.search(search_val='Primary School')['totalNumPages']
primary_sch = {'results':[]}
lat_longs = set()

try:
    for i in range(tot_page_num):
        for sch in client.search(search_val='Primary School',page_num=i+1)['results']:
            # Do not add coordinate if building is a specific block within a school
            if filter_school_search(sch['BUILDING'],primary_sch['results']):
                continue
            else:
                lat_long = tuple([float(sch['LATITUDE']), float(sch['LONGITUDE'])])
                if lat_long not in lat_longs:
                    lat_longs.add(lat_long)
                    primary_sch['results'].append({'name':sch['BUILDING'],'lat':float(sch['LATITUDE']),'long':float(sch['LONGITUDE'])})
except:
    print('No Results!')


In [21]:
len(primary_sch['results'])

156

In [22]:
with open('./amenities/primary_school_coords.json','w')as f:
    json.dump(primary_sch,f)

## Secondary/ High School /Institution

- search endpoint
    - keywords --> Secondary School, High School , 
    - Edge cases (Campus for both Secondary and College Students)
        - Raffles Institution, Hwa Chong Institution, Saint Joseph Institution, Anglo Chinese (Independent), Anglo Chinese (International)

In [26]:
tot_page_num = client.search(search_val='Secondary School')['totalNumPages']
sec_sch = {'results':[]}
lat_longs = set()

try:
    for i in range(tot_page_num):
        for sch in client.search(search_val='Secondary School',page_num=i+1)['results']:
            # Do not add coordinate if building is a specific block within a school
            if filter_school_search(sch['BUILDING'],sec_sch['results']):
                print(sch['BUILDING'])
                continue
            else:
                lat_long = tuple([float(sch['LATITUDE']), float(sch['LONGITUDE'])])
                if lat_long not in lat_longs:
                    lat_longs.add(lat_long)
                    sec_sch['results'].append({'name':sch['BUILDING'],'lat':float(sch['LATITUDE']),'long':float(sch['LONGITUDE'])})
except:
    print('No Results!')


BENDEMEER SECONDARY SCHOOL
WOODGROVE SECONDARY SCHOOL


In [27]:
tot_page_num = client.search(search_val='High School')['totalNumPages']

try:
    for i in range(tot_page_num):
        for sch in client.search(search_val='High School',page_num=i+1)['results']:
            # Do not add coordinate if building is a specific block within a school
            if filter_school_search(sch['BUILDING'],sec_sch['results']):
                print(sch['BUILDING'])
                continue
            else:
                lat_long = tuple([float(sch['LATITUDE']), float(sch['LONGITUDE'])])
                if lat_long not in lat_longs:
                    lat_longs.add(lat_long)
                    sec_sch['results'].append({'name':sch['BUILDING'],'lat':float(sch['LATITUDE']),'long':float(sch['LONGITUDE'])})
except:
    print('No Results!')

COMMIT LEARNING SCHOOLHOUSE @ CATHOLIC HIGH SCHOOL (PRIMARY)
NUS HIGH SCHOOL OF MATHEMATICS AND SCIENCE
CHINESE HIGH SCHOOL CLOCK TOWER BUILDING
CHUNG CHENG HIGH SCHOOL (MAIN) ADMINISTRATION BUILDING AND ENTRANCE ARCH
MARIS STELLA HIGH SCHOOL


In [28]:
len(sec_sch['results'])

131

In [29]:
with open('./amenities/secondary_school_coords.json','w')as f:
    json.dump(sec_sch,f)

## Junior Colleges

In [30]:
tot_page_num = client.search(search_val='Junior College')['totalNumPages']
jc = {'results':[]}
lat_longs = set()
try:
    for i in range(tot_page_num):
        for sch in client.search(search_val='Junior College',page_num=i+1)['results']:
            # Do not add coordinate if building is a specific block within a school
            if filter_school_search(sch['BUILDING'],jc['results']):
                print(sch['BUILDING'])
                continue
            else:
                lat_long = tuple([float(sch['LATITUDE']), float(sch['LONGITUDE'])])
                if lat_long not in lat_longs:
                    lat_longs.add(lat_long)
                    jc['results'].append({'name':sch['BUILDING'],'lat':float(sch['LATITUDE']),'long':float(sch['LONGITUDE'])})
except:
    print('No Results!')

CATHOLIC JUNIOR COLLEGE
EUNOIA JUNIOR COLLEGE


In [31]:
len(jc['results'])

13

In [32]:
with open('./amenities/college_coords.json','w')as f:
    json.dump(jc,f)

# Amenities

## National Parks

- queryname --> nationalparks
    - All places owned by National Parks Board
    - Open spaces, Playground, Fitness Corners, Parks, Nature Reserve
    - returns point type coordinate

- queryname --> nparks_parks
    - seems like return only nature parks and reserves
    - return polygon type coordinate

In [33]:
search_results = client.retrieve_theme(query_name='nationalparks')['SrchResults']
# search results starts from index
nparks_loc = {'results':[]}
lat_longs = set()
for npark in search_results[1:]:
    coord_list = npark['LatLng'].split(',')
    lat_long = tuple([float(coord_list[0]), float(coord_list[1])])
    if lat_long not in lat_longs:
        lat_longs.add(lat_long)
        nparks_loc['results'].append({'name':npark['NAME'],'lat':float(coord_list[0]),'long':float(coord_list[1])})

In [34]:
len(nparks_loc['results'])

430

In [37]:
with open('./amenities/npark_coords.json','w')as f:
    json.dump(nparks_loc,f)

## Sports

- retrieve_theme endpoint
    - queryname--> ssc_sports_facilities
        - returns point type coordinates
    - queryname --> sportsg_sport_facilities (This one seems more complete)
        - returns polygon type coordinates

In [39]:
search_results = client.retrieve_theme(query_name='ssc_sports_facilities')['SrchResults']
# search results starts from index
sports_loc = {'results':[]}
lat_longs = set()
for sport in search_results[1:]:
    coord_list = sport['LatLng'].split(',')
    lat_long = tuple([float(coord_list[0]), float(coord_list[1])])
    if lat_long not in lat_longs:
        lat_longs.add(lat_long)
        sports_loc['results'].append({'name':sport['NAME'],'lat':float(coord_list[0]),'long':float(coord_list[1])})

In [40]:
len(sports_loc['results'])

31

In [41]:
with open('./amenities/sport_facility_coords.json','w')as f:
    json.dump(sports_loc,f)

## Childcare

- queryname --> childcare
    - returns point type coordinate

In [43]:
search_results = client.retrieve_theme(query_name='childcare')['SrchResults']
# search results starts from index
childcare_loc = {'results':[]}
lat_longs = set()
for childcare in search_results[1:]:
    coord_list = childcare['LatLng'].split(',')
    lat_long = tuple([float(coord_list[0]), float(coord_list[1])])
    if lat_long not in lat_longs:
        lat_longs.add(lat_long)
        childcare_loc['results'].append({'name':childcare['NAME'],'lat':float(coord_list[0]),'long':float(coord_list[1])})

In [44]:
len(childcare_loc['results'])

1738

In [45]:
with open('./amenities/childcare_coords.json','w')as f:
    json.dump(childcare_loc,f)

## Cycling Paths

- queryname --> 'cyclingpath'
- returns polygon type coordinates

In [51]:
search_results = client.retrieve_theme(query_name='cyclingpath')['SrchResults']

In [52]:
cycling_path = {'results':[]}

for path in search_results[1:]:
    processed_latlng = []
    for coord in path['LatLng']:
        if coord[0]>90:
            processed_latlng.append([coord[1],coord[0]])
        else:
            processed_latlng.append([coord[0],coord[1]])
    cycling_path['results'].append({'name':path['NAME'],'latlng':processed_latlng})

In [53]:
with open('./amenities/cycling_path_coords.json','w')as f:
    json.dump(cycling_path,f)

## Disability Services

- returns point type coordinate

In [56]:
def remove_extra_spaces(input_string):
   return re.sub('\s+', ' ', input_string).strip()

In [59]:
search_results = client.retrieve_theme(query_name='disability')['SrchResults']
# search results starts from index
disability_loc = {'results':[]}
lat_longs = set()
for disability in search_results[1:]:
    coord_list = disability['LatLng'].split(',')
    lat_long = tuple([float(coord_list[0]), float(coord_list[1])])
    if lat_long not in lat_longs:
        lat_longs.add(lat_long)
        disability_loc['results'].append({'name':remove_extra_spaces(disability['NAME']),'lat':float(coord_list[0]),'long':float(coord_list[1])})

In [60]:
len(disability_loc['results'])

88

In [61]:
with open('./amenities/disability_service_coords.json','w')as f:
    json.dump(disability_loc,f)

## Eldercare Services

- returns point type coord

In [62]:
search_results = client.retrieve_theme(query_name='eldercare')['SrchResults']
# search results starts from index
eldercare_loc = {'results':[]}
lat_longs = set()
for eldercare in search_results[1:]:
    coord_list = eldercare['LatLng'].split(',')
    lat_long = tuple([float(coord_list[0]), float(coord_list[1])])
    if lat_long not in lat_longs:
        lat_longs.add(lat_long)
        eldercare_loc['results'].append({'name':eldercare['NAME'],'lat':float(coord_list[0]),'long':float(coord_list[1])})

In [63]:
len(eldercare_loc['results'])

120

In [64]:
with open('./amenities/eldercare_coords.json','w')as f:
    json.dump(eldercare_loc,f)

## Malls

In [66]:
# Filter away irrelevant results
def filter_mall_search(cur_mall_name,existing_mall_list):
   # do not add location of bank branches within a mall
   if 'OCBC' in cur_mall_name or 'CITIBANK' in cur_mall_name or 'DBS' in cur_mall_name or 'UOB' in cur_mall_name or 'MAYBANK' in cur_mall_name or 'STANDARD CHARTERED BANK' in cur_mall_name:
      return True

   # if name of search result is NIL do not add 
   if cur_mall_name == 'NIL':
      return True
   
   # if cur name already exists do not add
   for existing_mall in existing_mall_list:
      if cur_mall_name == existing_mall['name']:
         return True
   
   pattern = r"(\(|\[).*MALL.*(\]|\))|.*@.*MALL"
   if re.search(pattern, cur_mall_name):
      return True
   return False

In [67]:
tot_page_num = client.search(search_val='Mall')['totalNumPages']
malls_loc = {'results':[]}
lat_longs = set()

try:
    for i in range(tot_page_num):
        for mall in client.search(search_val='Mall',page_num=i+1)['results']:
            # do not add duplicate malls
            if not filter_mall_search(mall['BUILDING'],malls_loc['results']):
                lat_long = tuple([float(mall['LATITUDE']), float(mall['LONGITUDE'])])
                if lat_long not in lat_longs:
                    lat_longs.add(lat_long)
                    malls_loc['results'].append({'name':mall['BUILDING'],'lat':float(mall['LATITUDE']),'long':float(mall['LONGITUDE'])})
            else:
                # print those filtered
                print(mall['BUILDING'])
except:
    print('No Results!')


NIL
DBS WEST MALL
OCBC ESPLANADE MALL
UOB WEST MALL
UOB CITYLINK MALL
UOB WHITE SANDS MALL
UOB CITYLINK MALL
UOB WHITE SANDS MALL
CITIBANK MARINA BAY LINK MALL
DBS MARINA BAY LINK MALL
GROUND PLAZA (MARINA BAY LINK MALL)
DBS EASTPOINT MALL EBC 1
CITIBANK SUNTEC CITY MALL
OCBC EASTPOINT MALL - FAIRPRICE
UOB EASTPOINT MALL AUTO LOBBY
SUNTEC CITY MALL
CENTRAL MALL
UOB SUNTEC CITY MALL
DBS ESPLANADE MALL
OCBC TAMPINES MALL
UOB TAMPINES MALL
UOB TAMPINES MALL AUTOLOBBY
CENTRAL MALL
CENTRAL MALL
KEBUN BARU MALL
KEBUN BARU MALL
KEBUN BARU MALL
DBS BEDOK MALL 1
OCBC LOT 1 SHOPPERS MALL
DBS NTUC LOT 1 SHOPPERS' MALL
KEBUN BARU MALL
KEBUN BARU MALL
KEBUN BARU MALL
DBS NTUC HOUGANG MALL
DBS NTUC KANG KAR MALL
OCBC HOUGANG MALL BRANCH
UOB HOUGANG MALL
DBS HOUGANG MALL
DBS RIVERVALE MALL
UOB RIVERVALE MALL
OCBC RIVERVALE MALL
OCBC RIVERVALE MALL - FAIRPRICE
OCBC THE CLEMENTI MALL - CLEMENTI BRANCH
UOB CLEMENTI MALL
DBS NTUC CLEMENTI MALL
CITIBANK CLEMENTI MALL
OCBC THE CLEMENTI MALL - FAIRPRICE
STA

In [68]:
len(malls_loc['results'])

52

In [69]:
with open('./amenities/mall_coords.json','w')as f:
    json.dump(malls_loc,f)

## Hawker Centre

In [70]:
def filter_hawker_search(curr_hawker_name,existing_hawker_list):

   # if name of search result is NIL do not add 
   if curr_hawker_name == 'NIL':
      return True
   
   # if cur name already exists do not add
   for existing_hawker in existing_hawker_list:
      if curr_hawker_name == existing_hawker['name']:
         return True
   
   return False

In [71]:
tot_page_num = client.search(search_val='Hawker Centre')['totalNumPages']
hawker_loc = {'results':[]}
lat_longs = set()

try:
    for i in range(tot_page_num):
        for hawker in client.search(search_val='Hawker Centre',page_num=i+1)['results']:
            if not filter_hawker_search(hawker['BUILDING'],hawker_loc['results']):
                lat_long = tuple([float(hawker['LATITUDE']), float(hawker['LONGITUDE'])])
                if lat_long not in lat_longs:
                    lat_longs.add(lat_long)
                    hawker_loc['results'].append({'name':hawker['BUILDING'],'lat':float(hawker['LATITUDE']),'long':float(hawker['LONGITUDE'])})
            else:
                print(hawker['BUILDING'])
except:
    print('No Results!')


MARKET & HAWKER CENTRE (BLKS 2 & 3 CHANGI VILLAGE ROAD)
MARKET & HAWKER CENTRE (BLKS 20/21 MARSILING LANE)
HAWKER CENTRE (BLKS 79/79A CIRCUIT ROAD)


In [72]:
len(hawker_loc['results'])

53

In [74]:
with open('./amenities/hawker_coords.json','w')as f:
    json.dump(hawker_loc,f)